<a href="https://colab.research.google.com/github/xuxiufeng/Spam_ham_Classification/blob/main/Spam_ham_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unzip files

In [2]:
%%capture
!unzip enron1_test.zip
!unzip enron1_train.zip
!unzip enron4_test.zip
!unzip enron4_train.zip
!unzip hw1_test.zip
!unzip hw1_train.zip

# Naive Bayes

In [73]:
import sys
import collections
import os 
import re
import codecs
import numpy
import math

In [74]:
stop_words = ["a","about","above","after","again","against","all","am","an","and",
"any","are","aren't","as","at","be","because","been","before","being","below",
"between","both","but","by","can't","cannot","could","couldn't","did","didn't",
"do","does","doesn't","doing","don't","down","during","each","few","for","from",
"further","had","hadn't","has","hasn't","have","haven't","having","he","he'd",
"he'll","he's","her","here","here's","hers","herself","him","himself","his","how",
"how's","i","i'd","i'll","i'm","i've","if","in","into","is","isn't","it","it's","its",
"itself","let's","me","more","most","mustn't","my","myself","no","nor","not","of",
"off","on","once","only","or","other","ought","our","ours","ourselves","out","over",
"own","same","shan't","she","she'd","she'll","she's","should","shouldn't","so","some",
"such","than","that","that's","the","their","theirs","them","themselves","then","there",
"there's","these","they","they'd","they'll","they're","they've","this","those","through",
"to","too","under","until","up","very","was","wasn't","we","we'd","we'll","we're","we've",
"were","weren't","what","what's","when","when's","where","where's","which","while","who",
"who's","whom","why","why's","with","won't","would","wouldn't","you","you'd","you'll",
"you're","you've","your","yours","yourself","yourselves"]

def read_file(file,file_path):
    fileHandler = codecs.open(file_path+"/" + file,'rU','latin-1')
    Findwords = re.findall('[A-Za-z0-9\']+', fileHandler.read())
    allwords = list()
    for word in Findwords:
        word = word.lower()
        allwords+=[word]
    fileHandler.close()    
    return allwords
    
def GetWordListsAndNumberOffiles(file_path):
    wordList = list()
    NumberOfFiles = 0
    for files in os.listdir(file_path):
        if files.endswith(".txt"):
            wordList += read_file(files,file_path)
            NumberOfFiles+=1
    return wordList, NumberOfFiles

def initiliazeMatrix(row, column):
    featureMatrix = [0] * row
    for i in range(row):
        featureMatrix[i] = [0] * column
    return featureMatrix

In [75]:
train_path = 'enron1/train'

In [76]:
#location of the folder for ham & spam for train and test

HamFolderPath = train_path + '/ham'
SpamFolderPath = train_path + '/spam'
    

Find out all the words in ham folder and spam folder and find there counts 

In [77]:
NumberOfHam, NumberOfSpam = 0, 0
WordListInham = []
WordListInspam = []
WordListInham,NumberOfHam = GetWordListsAndNumberOffiles(HamFolderPath)
WordListInspam,NumberOfSpam = GetWordListsAndNumberOffiles(SpamFolderPath)

Function to Find out P(ham) and P(spam), by calculating the number of ham/spam documents and total number of documents

In [78]:
def FindPHamOrSpam(HamOrSpam):
    if HamOrSpam == "spam":
        Pspam = NumberOfSpam/(NumberOfSpam + NumberOfHam)
        return Pspam
    else:
        Pham = NumberOfHam/(NumberOfSpam + NumberOfHam)
        return Pham

After finding all the words in ham and spam files , we will find the distinct words and its count

In [79]:
HamDictionary = dict(collections.Counter(w.lower() for w in WordListInham))
SpamDictionary = dict(collections.Counter(w.lower() for w in WordListInspam))

making bag of words for both ham and spam and further counting the count of each Distinct word in it

In [80]:
bagOfWords = WordListInham + WordListInspam
BagOfWordsDict = collections.Counter(bagOfWords)

In [81]:
def UpdateCountOfMissingWords(AllWords,HamSpamWords):
    for words in AllWords:
        if words not in HamSpamWords:
            HamSpamWords[words] = 0

getting missing words in each Ham and Spam list and adding them and intializing their count= 0

In [82]:
UpdateCountOfMissingWords(BagOfWordsDict,HamDictionary)
UpdateCountOfMissingWords(BagOfWordsDict,SpamDictionary)

In [83]:
ProbOfHamWords = dict()
ProbOfSpamWords = dict()

In [84]:
def FindProbabilityOfWord(classifier,removestopwords):
    Counter = 0
    if(removestopwords ==1):
            for word in stop_words:
                if word in HamDictionary:
                    del HamDictionary[word]
                if word in SpamDictionary:
                    del SpamDictionary[word]
                if word in BagOfWordsDict:
                    del BagOfWordsDict[word]                    
    if classifier == "ham":
        for word in HamDictionary:
            Counter += (HamDictionary[word] + 1)
        for word in HamDictionary:
            ProbOfHamWords[word] = math.log((HamDictionary[word] + 1)/Counter ,2)
    elif classifier == "spam":
        for word in SpamDictionary:
            Counter += (SpamDictionary[word] + 1)
        for word in SpamDictionary:
            ProbOfSpamWords[word] = math.log((SpamDictionary[word] + 1)/Counter ,2) 

caluculating probability for each word in ham and Spam folders 

In [85]:
FindProbabilityOfWord("ham",0)
FindProbabilityOfWord("spam",0) 

Finally classify the emails as ham or spam   

In [86]:
def PredictHamOrSpam(pathToFile, classifier):
    ProbabilityOfHam = 0 
    ProbabilityOfSpam = 0 
    InCorrectlyClassified = 0
    NumberOfFiles = 0
                   
    if classifier == "spam":
        for fileName in os.listdir(pathToFile):
            words =read_file(fileName,pathToFile)
            #find actual P(ham) and P(spam) i.e. (number of ham documents / Total no of documents)
            ProbabilityOfHam = math.log(FindPHamOrSpam("ham"),2)
            ProbabilityOfSpam = math.log(FindPHamOrSpam("spam"),2)
            #log(P(ham|bodyText)) = log(P(ham)) + log(P(word1|ham)) + log(P(word2|ham)) + .... 
            for word in words:
                if word in ProbOfHamWords:
                    ProbabilityOfHam += ProbOfHamWords[word]
                if word in ProbOfSpamWords:
                    ProbabilityOfSpam += ProbOfSpamWords[word]
            NumberOfFiles +=1
            if(ProbabilityOfHam >= ProbabilityOfSpam):
                InCorrectlyClassified+=1
    if classifier == "ham":
        for fileName in os.listdir(pathToFile):
            words =read_file(fileName,pathToFile)
            #find actual P(ham) and P(spam) i.e. (number of ham documents / Total no of documents)
            ProbabilityOfHam = math.log(FindPHamOrSpam("ham"),2)
            ProbabilityOfSpam = math.log(FindPHamOrSpam("spam"),2)
            #log(P(ham|bodyText)) = log(P(ham)) + log(P(word1|ham)) + log(P(word2|ham)) + ....            
            for word in words:
                if word in ProbOfHamWords:
                    ProbabilityOfHam += ProbOfHamWords[word]
                if word in ProbOfSpamWords:
                    ProbabilityOfSpam += ProbOfSpamWords[word]
            NumberOfFiles +=1
            if(ProbabilityOfHam <= ProbabilityOfSpam):
                InCorrectlyClassified+=1
    return InCorrectlyClassified,NumberOfFiles 

print("Executing Naive Bayes with stop words for Ham & Spam test emails :")  


Executing Naive Bayes with stop words for Ham & Spam test emails :


In [87]:
test_path = 'enron1/test'

In [88]:
HamTestPath = test_path + '/ham'
SpamTestPath = test_path + '/spam'        
IncorrectlyClassifiedHam,TotalHamEmails = PredictHamOrSpam(HamTestPath, "ham")
IncorrectlyClassifiedSpam,TotalSpamEmails = PredictHamOrSpam(SpamTestPath,"spam")
AccuracyOfHamClassification = round(((TotalHamEmails - IncorrectlyClassifiedHam )/(TotalHamEmails ))*100,2)
AccuracyOfSpamClassification = round(((TotalSpamEmails -  IncorrectlyClassifiedSpam )/(TotalSpamEmails))*100,2)
AllEmailClassified = TotalHamEmails + TotalSpamEmails
TotalIncorrectClassified = IncorrectlyClassifiedHam + IncorrectlyClassifiedSpam
OverAllAccuracy = round(((AllEmailClassified  - TotalIncorrectClassified )/AllEmailClassified)*100,2)

In [89]:
print("\nTotal number of files: ", AllEmailClassified)
print("\nCalculating Accuracy over Ham Emails")
print("Total number of Ham Emails: ", TotalHamEmails)
print("Number of Emails Classified as Ham: ", TotalHamEmails - IncorrectlyClassifiedHam)
print("Number of Emails Classified as Spam: ",IncorrectlyClassifiedHam)
print("\nNaive Bayes Accuracy For Ham Emails Classification:" + str(AccuracyOfHamClassification) + "%")


Total number of files:  456

Calculating Accuracy over Ham Emails
Total number of Ham Emails:  307
Number of Emails Classified as Ham:  300
Number of Emails Classified as Spam:  7

Naive Bayes Accuracy For Ham Emails Classification:97.72%


In [90]:
print("\nCalculating Accuracy over Spam Emails")
print("Total number of Spam Emails: ", TotalSpamEmails)
print("Number of Emails Classified as Spam: ", TotalSpamEmails - IncorrectlyClassifiedSpam)
print("Number of Emails Classified as Ham: ",IncorrectlyClassifiedSpam)
print("\nNaive Bayes Accuracy For Spam Emails Classification: " + str(AccuracyOfSpamClassification) + "%") 


Calculating Accuracy over Spam Emails
Total number of Spam Emails:  149
Number of Emails Classified as Spam:  127
Number of Emails Classified as Ham:  22

Naive Bayes Accuracy For Spam Emails Classification: 85.23%


In [91]:
print("\nNaive Bayes Total accuracy for Test Emails: " + str(OverAllAccuracy) + "%")

print("\n")


Naive Bayes Total accuracy for Test Emails: 93.64%




In [92]:
print("Executing Naive Bayes after removing stop words")
FindProbabilityOfWord("ham",1)
FindProbabilityOfWord("spam",1) 

Executing Naive Bayes after removing stop words


In [93]:
IncorrectlyClassifiedHam,TotalHamEmails = PredictHamOrSpam(HamTestPath, "ham")
IncorrectlyClassifiedSpam,TotalSpamEmails = PredictHamOrSpam(SpamTestPath,"spam")
AccuracyOfHamClassification = round(((TotalHamEmails - IncorrectlyClassifiedHam )/(TotalHamEmails ))*100,2)
AccuracyOfSpamClassification = round(((TotalSpamEmails -  IncorrectlyClassifiedSpam )/(TotalSpamEmails))*100,2)
AllEmailClassified = TotalHamEmails + TotalSpamEmails
TotalIncorrectClassified = IncorrectlyClassifiedHam + IncorrectlyClassifiedSpam
OverAllAccuracy = round(((AllEmailClassified  - TotalIncorrectClassified )/AllEmailClassified)*100,2)

In [94]:
print("\nTotal number of files: ", AllEmailClassified)
print("\nCalculating Accuracy over Ham Emails")
print("Total number of Ham Emails: ", TotalHamEmails)
print("Number of Emails Classified as Ham: ", TotalHamEmails - IncorrectlyClassifiedHam)
print("Number of Emails Classified as Spam: ",IncorrectlyClassifiedHam)
print("\nNaive Bayes Accuracy For Ham Emails Classification:" + str(AccuracyOfHamClassification) + "%")



Total number of files:  456

Calculating Accuracy over Ham Emails
Total number of Ham Emails:  307
Number of Emails Classified as Ham:  301
Number of Emails Classified as Spam:  6

Naive Bayes Accuracy For Ham Emails Classification:98.05%


In [95]:
print("\nCalculating Accuracy over Spam Emails")
print("Total number of Spam Emails: ", TotalSpamEmails)
print("Number of Emails Classified as Spam: ", TotalSpamEmails - IncorrectlyClassifiedSpam)
print("Number of Emails Classified as Ham: ",IncorrectlyClassifiedSpam)
print("\nNaive Bayes Accuracy For Spam Emails Classification: " + str(AccuracyOfSpamClassification) + "%")


Calculating Accuracy over Spam Emails
Total number of Spam Emails:  149
Number of Emails Classified as Spam:  122
Number of Emails Classified as Ham:  27

Naive Bayes Accuracy For Spam Emails Classification: 81.88%


In [96]:
print("\nNaive Bayes Total accuracy for Test Emails: " + str(OverAllAccuracy) + "%")

print("\n")


Naive Bayes Total accuracy for Test Emails: 92.76%


